In [1]:
%config IPCompleter.greedy=True
import re
import json
import csv
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm
from elasticsearch import Elasticsearch
from elasticsearch.helpers import parallel_bulk
from time import time
import pandas as pd

In [2]:
def json_read(filename):
    with open(filename, 'r') as inf:
        res = json.load(inf)
    return res

def json_dump(obj, filename, ea=False, indent=4):
    with open(filename, 'w') as ouf:
        json.dump(obj, ouf, ensure_ascii=ea, indent=indent)

### Prepare toloka tasks

#### Read matches data

In [60]:
items = json_read('answers/matches_1_5000.json')

In [54]:
# for question matching only
edited_items = []
for item in items:
    q = item['text']
    a = item['answer']
    m = item['matches']
    edited_items.append({
        'question': q,
        'answer': a,
        'matches': m
    })
items = edited_items

In [61]:
# for answer matching only
edited_items = []
for item in items:
    a = item['text']
    q = item['question']
    m = item['matches']
    edited_items.append({
        'question': q,
        'answer': a,
        'matches': m
    })
items = edited_items

In [62]:
for item in items:
    q = item['question']
    a = item['answer']
    if f'({a})' in q:
        q = q.replace(f'({a})', '')
        item['question'] = q

In [56]:
# for question matching only
for item in tqdm(items):
    old_matches = item['matches']
    new_matches = []
    for match in old_matches:
        if match['source'] == 'phrase':
            new_matches.append(match)
    for match in old_matches:
        if match['source'] == 'single':
            new_matches.append(match)
    for match in old_matches:
        if match['source'] == 'fulltext':
            new_matches.append(match)
    assert len(old_matches) == len(new_matches)
    item['matches'] = new_matches

In [44]:
items[:30]

[{'question': 'Сколько существует законов Ньютона?',
  'answer': '3',
  'matches': [{'qid': 'Q935',
    'label': 'Ньютон',
    'score': 12.546672,
    'name': 'Исаак Ньютон',
    'description': 'английский физик, математик и астроном, один из создателей классической физики',
    'ruwiki': 'Ньютон, Исаак',
    'views': 13196,
    'source': 'single'},
   {'qid': 'Q38433',
    'label': 'Законы Ньютона',
    'score': 19.39318,
    'name': 'Законы Ньютона',
    'description': None,
    'ruwiki': 'Законы Ньютона',
    'views': 17927,
    'source': 'fulltext'},
   {'qid': 'Q134465',
    'label': 'Закон тяготения Ньютона',
    'score': 16.19127,
    'name': 'Классическая теория тяготения Ньютона',
    'description': None,
    'ruwiki': 'Классическая теория тяготения Ньютона',
    'views': 4371,
    'source': 'fulltext'},
   {'qid': 'Q122508',
    'label': 'Первый закон Ньютона',
    'score': 16.19127,
    'name': 'Инерция',
    'description': 'Свойства тел сохранять прямолинейное и равномерное

In [63]:
json_dump(items, 'answers/matches_1_5000.json')

#### Filter questions

In [19]:
def is_number(s):
    res = True
    try:
        float(s.replace(',', '.'))
    except:
        res = False
    return res

def has_multichoice(q):
    mc_words = ['из этого', 'из перечисл', 'из этих', 'из следующ']
    for mcw in mc_words:
        if mcw in q:
            return True
    return False

In [20]:
def filter_numbers_in_answer(item):
    return not is_number(item['answer'])

def filter_multichoice_in_question(item):
    return not has_multichoice(item['question'])

def filter_too_many_matches(item):
    return len(item['matches']) < 16

In [45]:
cnt = 0
for item in items:
    if not filter_too_many_matches(item):
        print(item['question'])
        cnt += 1
cnt

Какой современный русский художник в живописи и графике создал образы своих современников: Репина, Коровина, Левитана, Горького, Чехова, Шаляпина, Станиславского, Качалова, Ермоловой, Римского-Корсакова, Лескова?
Как называла Софья Саввишна Попова мужа своего, Льва Ивановича в рассказе А. П. Чехова "Житейские невзгоды"?
Какая профессия объединяет Иисуса Навина, Вольтера, Даниэля Дефо, Бомарше, Джакомо Казанову и Д. И. Менделеева?
Какая трапеза сердечно связывала Альфонса Доде, Эдмона Гонкура, Эмиля Золя и Ивана Тургенева?
Что объединяет таких досточтимых мужей как Иммануил Кант, Леонардо да Винчи, Микеланджело Буонаротти, Людвиг ван Бетховен, Исаак Ньютон, Иван Тургенев и других?
Какой плохой воздух погубил Александра Македонского, Марка Аврелия, Альбрехта Дюрера и одну из армий Наполеона?
Что общего между Анголой, Бразилией, Гвинеей-Бисау, Кабо-Верде, Макао, Мозамбиком и Сан-Томе и Принсипи?


7

In [46]:
cnt = 0
for item in items:
    if not filter_multichoice_in_question(item):
        print(item['question'])
        cnt += 1
cnt

Какие из этих часов могут иметь секундную стрелку?
Что из перечисленного является спортивным снарядом?
Какое из этих украшений можно встретить на новогодней ёлке?
Месторасположение какой из этих станций в Антарктиде совпадает с полюсом холода?
Кто из этих персонажей варил кашу из топора?
Что из этого фрукт?
Какая из этих птиц умеет летать? 
Какая из перечисленных программ является браузером?
Какого из этих цветов нет на флаге Болгарии?
Какой из этих видов спорта - олимпийский? 
Кто из этих американских президентов был убит?
Какой из этих темпов в музыке - самый быстрый? 
В каком из следующих продуктов питания наибольшее содержание алюминия? 
Чья должность из этих членов экипажа судна самая низшая?
Какой из этих праздничных дней отмечается позднее других?
Какое из этих женских имен - греческого происхождения?
Какой из этих писателей написал "Азбуку" и "Новую Азбуку"?
Кто из перечисленных великих князей стал первым русским царём?
Кто из этих поэтов прожил самую короткую жизнь?
Какое из э

158

In [47]:
items_filtered = []
for item in items:
    if filter_multichoice_in_question(item) and \
       filter_too_many_matches(item):
        items_filtered.append(item)

In [48]:
len(items_filtered)

4835

In [49]:
max([len(item['matches']) for item in items_filtered])

15

#### Create record from data

In [26]:
def match_to_task(match, i):
    res = {}
    
    name = match['name'] if match['name'] else '???'
    if match['description']:
        if len(match['description']) > 50:
            desc = match['description'][:47] + '...'
        else:
            desc = match['description']
        name += f' ({desc})'
    name = name.replace('"', '\\"').replace(',', '\,')
    res['text'] = name
    
    if match['ruwiki']:
        page = match['ruwiki'].replace(' ', '_')
        url = f'https://ru.wikipedia.org/wiki/{page}'.replace(',', '\,')
        res['link_text'] = 'Википедия'
    else:
        page = match['qid']
        url = f'https://www.wikidata.org/wiki/{page}'.replace(',', '\,')
        res['link_text'] = 'Викиданные'
    url = url.replace('"', '\\"')
    res['url'] = url
    
    res['value'] = i
    res['hotkey'] = str(i)
    
    return res

def data_to_task(data):
    task = []
    
    # question
    task.append(data['question'])
    
    # answer
    task.append(data['answer'])
    
    # matches
    matches = data['matches']
    task_matches = []
    for i, match in enumerate(matches):
        cur_task_match = match_to_task(match, i + 1)
        task_matches.append('{' + '\,'.join([f'"{k}":"{v}"' for k, v in cur_task_match.items()]) + '}')
    task.append(','.join(task_matches))
    
    # golden
    task.append('')
    
    # hint
    task.append('')
    
    return task

#### Create tsv file

In [50]:
header = ['INPUT:question', 'INPUT:answer', 'INPUT:matches', 'GOLDEN:q_matches', 'HINT:text']

with open('q_task_1001_5000.tsv', 'w') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t', lineterminator='\n', quotechar='"')
    writer.writerow(header)
    for item in items_filtered[1001:]:
        writer.writerow(data_to_task(item))

### Learning samples

In [3]:
samples = pd.read_csv('tasks_263320_TRAINING_20-12-2019.tsv', sep='\t', header=0)

In [4]:
samples

,INPUT:answer,INPUT:matches,INPUT:question,GOLDEN:q_matches,HINT:text,TASK:id,TASK:overlap,TASK:assignments_count,TASK:correct_answers_percent
0,Наф-Наф,"""[{\\""url\\"":\\""https://ru.wikipedia.org/wiki/...","Чей домик оказался крепче всех в сказке ""3 пор...","""{1=false, 2=false, 3=false, 4=false, 5=false,...","Нужно выбрать вариант 7, так как именно эта ск...",0000040498--5dfa604457180300e7767b41,infinite,0,NaN
1,Атеист,"""[{\\""url\\"":\\""https://ru.wikipedia.org/wiki/...",Как называется неверующий человек?,"""{1=false, 2=false, 3=false, 4=false, 5=false,...","Перейдя на Википедию, выясняем, что вариант 1 ...",0000040498--5dfa604457180300e7767b43,infinite,0,NaN
2,Телевизор,"""[{\\""url\\"":\\""https://ru.wikipedia.org/wiki/...","Что, по мнению почтальона Печкина, является гл...","""{1=false, 2=false, 3=false, 4=false, 5=false,...","Нужно выбрать вариант 8, так как он соответств...",0000040498--5dfa604457180300e7767b5d,infinite,0,NaN
3,Террариум,"""[{\\""url\\"":\\""https://ru.wikipedia.org/wiki/...",Как называется отгороженное пространство для с...,"""{1=true, 2=false, 3=false, 4=false, 5=false, ...",Вариант 1 точно нужно выбрать - про земноводны...,0000040498--5dfa604457180300e7767b77,infinite,0,NaN
4,Тычинка,"""[{\\""url\\"":\\""https://ru.wikipedia.org/wiki/...","Как называется мужской орган цветка, состоящий...","""{1=true, 2=false, 3=false, 4=false, 5=false, ...",Подходит только вариант 1.,0000040498--5dfa604457180300e7767b97,infinite,0,NaN
5,3600,"""[{\\""url\\"":\\""https://ru.wikipedia.org/wiki/...",Сколько секунд в часе?,"""{1=true, 2=false, 3=false, 4=false, 5=true, 6...",В вопросе упоминаются две единицы измерения вр...,0000040498--5dfa604457180300e7767ba2,infinite,0,NaN
6,"Детское Село, в 1937 - Пушкин","""[{\\""url\\"":\\""https://ru.wikipedia.org/wiki/...",Какое название в 1918 году дали городу Царское...,"""{1=true, 2=false, 3=false, 4=false, 5=false, ...","Несмотря на то, что в вопросе город упоминаетс...",0000040498--5dfa604457180300e7767bae,infinite,0,NaN
7,Октябрь,"""[{\\""text\\"":\\""десятый\\""\,\\""link_text\\"":\...",Какой месяц идет десятым?,"""{1=true, 2=false, 3=false, 4=true, 5=false, 6...",Подходят варианты 1 и 4 - вариант 1 соответств...,0000040498--5dfcc109c0696e00d6495794,infinite,0,NaN
8,Нинель,"""[{\\""text\\"":\\""Иосиф Давыдович Кобзон (совет...",Как зовут жену Иосифа Кобзона?,"""{1=true, 2=false, 3=false, 4=false, 5=false, ...","Подходит вариант 1. Обратите внимание, что не ...",0000040498--5dfcc109c0696e00d6495795,infinite,0,NaN
9,Белый дом,"""[{\\""text\\"":\\""Президентские выборы в США в ...",Как называется официальная резиденция Президен...,"""{1=false, 2=false, 3=true, 4=false, 5=true, 6...",В вопросе речь идет об официальной резиденции....,0000040498--5dfcc109c0696e00d64957a4,infinite,0,NaN


In [43]:
st = samples.loc[:, ['INPUT:answer','INPUT:question']]
st

,INPUT:answer,INPUT:question
0,Наф-Наф,"Чей домик оказался крепче всех в сказке ""3 пор..."
1,Атеист,Как называется неверующий человек?
2,Телевизор,"Что, по мнению почтальона Печкина, является гл..."
3,Террариум,Как называется отгороженное пространство для с...
4,Тычинка,"Как называется мужской орган цветка, состоящий..."
5,3600,Сколько секунд в часе?
6,"Детское Село, в 1937 - Пушкин",Какое название в 1918 году дали городу Царское...
7,Октябрь,Какой месяц идет десятым?
8,Нинель,Как зовут жену Иосифа Кобзона?
9,Белый дом,Как называется официальная резиденция Президен...


In [44]:
# st.to_csv('learning_task.csv', sep='\t', index=False)
st.to_csv('q_learning.csv', sep=',', index=True)

In [5]:
poses = pd.read_csv('q_learning.csv', header=0)
poses['pos']

0      2
1     12
2     14
3      6
4      0
5     19
6     18
7     17
8      7
9     16
10    10
11     4
12     1
13    11
14    15
15     5
16     9
17     8
18    13
19     3
Name: pos, dtype: int64

In [6]:
learning_samples = samples.loc[poses['pos'], 'INPUT:answer':'HINT:text']

In [7]:
all_tasks = pd.read_csv('q_task_1_500.tsv', sep='\t', header=0)
all_tasks[all_tasks['INPUT:question'] == 'Какой специалист использует горн?']

,INPUT:question,INPUT:answer,INPUT:matches,GOLDEN:q_matches,HINT:text
4,Какой специалист использует горн?,Кузнец,"{""text"":""Специалист (фильм 1994 года США)""\,""l...",NaN,NaN


In [8]:
def convert_output_json(pseudo_json_str):
    json_pairs = pseudo_json_str[2:-2].split(', ')
    json_obj = {}
    for pair in json_pairs:
        opt, res = pair.split('=')
        res = True if res == 'true' else False
        json_obj[opt] = res
    return json.dumps(json_obj)

In [9]:
convert_output_json('"{1=false, 2=false, 3=false, 4=false, 5=false, 6=false, 7=true, 8=false}"')

'{"1": false, "2": false, "3": false, "4": false, "5": false, "6": false, "7": true, "8": false}'

In [10]:
for i, ls in learning_samples.iterrows():
    q = ls['INPUT:question']
    pseudo_json = ls['GOLDEN:q_matches']
    serialized_matches = all_tasks[all_tasks['INPUT:question'] == q]['INPUT:matches'].values[0]
    learning_samples.at[i, 'INPUT:matches'] = serialized_matches
    learning_samples.at[i, 'GOLDEN:q_matches'] = convert_output_json(pseudo_json)

In [11]:
learning_samples

,INPUT:answer,INPUT:matches,INPUT:question,GOLDEN:q_matches,HINT:text
2,Телевизор,"{""text"":""Меню «Пуск»""\,""link_text"":""Википедия""...","Что, по мнению почтальона Печкина, является гл...","{""1"": false, ""2"": false, ""3"": false, ""4"": fals...","Нужно выбрать вариант 8, так как он соответств..."
12,Такова жизнь,"{""text"":""C’est la vie""\,""link_text"":""Википедия...","Как переводится с французского фраза: ""C est l...","{""1"": false, ""2"": false, ""3"": false, ""4"": fals...",Ни один из вариантов не подходит. Варианты 1 и...
14,Перчатку,"{""text"":""Братья-сёстры\, соперники-соперницы""\...","Что бросают, вызывая соперника на дуэль?","{""1"": false, ""2"": false, ""3"": true, ""4"": false...","Подходит только вариант 3, варианты 4-8 соотве..."
6,"Детское Село, в 1937 - Пушкин","{""text"":""Пушкин (город в составе Санкт-Петербу...",Какое название в 1918 году дали городу Царское...,"{""1"": true, ""2"": false, ""3"": false, ""4"": false...","Несмотря на то, что в вопросе город упоминаетс..."
0,Наф-Наф,"{""text"":""Сказка сказок (мультфильм)""\,""link_te...","Чей домик оказался крепче всех в сказке ""3 пор...","{""1"": false, ""2"": false, ""3"": false, ""4"": fals...","Нужно выбрать вариант 7, так как именно эта ск..."
19,В париже,"{""text"":""Мост Монкальм""\,""link_text"":""Википеди...",В каком городе находится Эйфелева башня?,"{""1"": false, ""2"": false, ""3"": false, ""4"": fals...",Подходит только вариант 8.
18,Гаврюша,"{""text"":""Простоквашино (деревня в Тонкинском р...",Как кот Матроскин из Простоквашина назвал свое...,"{""1"": false, ""2"": false, ""3"": false, ""4"": fals...",Подходит только вариант 8. Вариант 7 не подход...
17,Меркурий,"{""text"":""Земля (третья от Солнца планета Солне...",Какая планета ближе всех находится к Солнцу?,"{""1"": false, ""2"": false, ""3"": true, ""4"": false...",Подходят варианты 3 и 8 - в вопросе упомянуты ...
7,Октябрь,"{""text"":""десятый""\,""link_text"":""Викиданные""\,""...",Какой месяц идет десятым?,"{""1"": true, ""2"": false, ""3"": false, ""4"": true,...",Подходят варианты 1 и 4 - вариант 1 соответств...
16,ВМФ,"{""text"":""род войск (составная часть вооружённы...",Какими войсками командует адмирал?,"{""1"": false, ""2"": false, ""3"": true, ""4"": false...",Необходимо выбрать вариант 3. Варианты 4 и 7 в...


In [12]:
learning_samples.to_csv('q_learning_task.csv', sep='\t', index=False)

In [236]:
trtr = json_read(f'{path}quiz_answer_entities_data_1_5000.json')
len(trtr)

NameError: name 'path' is not defined

In [132]:
'00008077ea--5dcc63f2e48f45011a3f98f7	"Юрмала"	[{"url":"https://ru.wikipedia.org/wiki/Юрмала","text":"Юрмала (город в Латвии)","value":"1","hotkey":"1","link_text":"Википедия"},{"url":"https://ru.wikipedia.org/wiki/Юрмала_(конкурс)","text":"Юрмала (конкурс молодых исполнителей песни в СССР)","value":"2","hotkey":"2","link_text":"Википедия"},{"url":"https://ru.wikipedia.org/wiki/Юрмала_(аэропорт)","text":"Юрмала","value":"3","hotkey":"3","link_text":"Википедия"},{"url":"https://ru.wikipedia.org/wiki/Юрмала_(футбольный_клуб)","text":"Юрмала","value":"4","hotkey":"4","link_text":"Википедия"},{"url":"https://ru.wikipedia.org/wiki/Юрман_(приток_Сылвы)","text":"Юрман","value":"5","hotkey":"5","link_text":"Википедия"}]	Какой музыкальный конкурс принес известность Александру Малинину?	2	99.99%		'.split('\t')

['00008077ea--5dcc63f2e48f45011a3f98f7',
 '"Юрмала"',
 '[{"url":"https://ru.wikipedia.org/wiki/Юрмала","text":"Юрмала (город в Латвии)","value":"1","hotkey":"1","link_text":"Википедия"},{"url":"https://ru.wikipedia.org/wiki/Юрмала_(конкурс)","text":"Юрмала (конкурс молодых исполнителей песни в СССР)","value":"2","hotkey":"2","link_text":"Википедия"},{"url":"https://ru.wikipedia.org/wiki/Юрмала_(аэропорт)","text":"Юрмала","value":"3","hotkey":"3","link_text":"Википедия"},{"url":"https://ru.wikipedia.org/wiki/Юрмала_(футбольный_клуб)","text":"Юрмала","value":"4","hotkey":"4","link_text":"Википедия"},{"url":"https://ru.wikipedia.org/wiki/Юрман_(приток_Сылвы)","text":"Юрман","value":"5","hotkey":"5","link_text":"Википедия"}]',
 'Какой музыкальный конкурс принес известность Александру Малинину?',
 '2',
 '99.99%',
 '',
 '']

In [65]:
aggs = [
    pd.read_csv('answers/aggregated_results_500_5000.tsv', sep='\t'),
    pd.read_csv('answers/aggregated_results_5001_10000.tsv', sep='\t'),
    pd.read_csv('answers/aggregated_results_10001_14000.tsv', sep='\t'),
    pd.read_csv('answers/aggregated_results_14001_end.tsv', sep='\t')
]
agg = pd.concat(aggs)

In [66]:
agg.head()

,SYSTEM:task_id,INPUT:answer,INPUT:matches,INPUT:question,OUTPUT:match,CONFIDENCE:match,GOLDEN:match,Unnamed: 7
0,00007b963a--5dcae89b4d462b0118848f41,Христофор Колумб,"[{""url"":""https://ru.wikipedia.org/wiki/Колумб,...","Кто собирался открыть Индию, а открыл Америку?",1,99.99%,NaN,NaN
1,00007b963a--5dcae89b4d462b0118848f42,Шиповник,"[{""url"":""https://ru.wikipedia.org/wiki/Шиповни...","Какое растение называют ""дикой розой""?",1,100.00%,1.0,NaN
2,00007b963a--5dcae89b4d462b0118848f43,Рыба!,"[{""url"":""https://ru.wikipedia.org/wiki/Рыбы"",""...","Каким словом нередко завершается процесс ""заби...",0,98.82%,NaN,NaN
3,00007b963a--5dcae89b4d462b0118848f44,Зеленый,"[{""url"":""https://ru.wikipedia.org/wiki/Зелёный...",Какого цвета минерал малахит?,0,99.99%,NaN,NaN
4,00007b963a--5dcae89b4d462b0118848f45,Академик,"[{""url"":""https://ru.wikipedia.org/wiki/Академи...",Какое звание является наивысшим в научной иера...,1,99.98%,NaN,NaN


In [67]:
len(agg[agg['OUTPUT:match'] != 0])

12013

In [145]:
agg['OUTPUT:match'].value_counts()[0] / len(agg)

0.16790089279534917

NameError: name 'pd' is not defined